In [7]:
import numpy as np
import pandas as pd
import random
import estimator as est
from optimizer import OptVariance
np.set_printoptions(precision=3)

# set key parameters
d = 20
dist_type = "exp"
est_type = "sw"
N=100000
total_repeat = 10

# set x
x_max = 1
x_step_size = x_max/d
x_grid = x_step_size*np.arange(d)
print("x=", x_grid)

# set q(x)
if dist_type=='exp':
    x_q = np.exp(-3*x_grid)
    x_q = x_q/np.sum(x_q)
elif dist_type=='uni':
    x_q = np.ones(d)/d
else:
    print('warning: invaild distribution type!')
print("q(x)=",x_q)

# save data
temp = {'x_grid': x_grid, 'x_q':x_q}
filename = "data/%s_q_%d.csv" %(dist_type,d)
pd.DataFrame(temp).to_csv(filename)

# generated data samples
idx_original = random.choices(np.arange(d),x_q,k=N)
sample_original = x_grid[idx_original]
print('%d data samples generated with %s distribution.\n' %(N, dist_type))

# set epsilon
eps_grid = 0.5*np.arange(1,6)
filename = 'data/eps_grid.csv'
temp = {'eps_grid': eps_grid}
pd.DataFrame(temp).to_csv(filename)

for i in range(len(eps_grid)):
    eps= eps_grid[i]
    print('eps=%.2f'%(eps))

    if est_type == 'sw':
        a_grid, M = est.SquareWave(eps,x_grid)
    elif est_type == 'grr':
        a_grid, M = est.GenRandResp(eps, x_grid)
    else:
        print('warning: invalid estimator type!')
    print('Found matrix M for %s estimator!' %(est_type))

    filename = 'data/%s_M_%.2f_%d.csv'%(est_type,eps,d)
    pd.DataFrame(M).to_csv(filename)
    filename = 'data/%s_a_%.2f_%d.csv'%(est_type,eps,d)
    temp = {'a_grid':a_grid}
    pd.DataFrame(temp).to_csv(filename)

    rand_num = np.zeros((N,d))
    for j in range(d):
        rand_num[:,j] = random.choices(a_grid,M[:,j],k=N)
    print("Random num pools generated.")

    x_q_noisy = np.zeros((1,d))
    for k in range(total_repeat):
        idx_noise = random.choices(range(N),k=N)
        idx_pair = list(zip(idx_noise,idx_original))
        sample_perturbed = rand_num[tuple(zip(*idx_pair))]
        elements,counts = np.unique(sample_perturbed,return_counts=True)
        x_q_est = est.EM(M,counts,eps)
        x_q_noisy +=x_q_est
    x_q_noisy = x_q_noisy/np.sum(x_q_noisy)
    print('Distributed Estimation with %s complete with %d repeats' 
        %(est_type, total_repeat))
    print('q_est(x)=', x_q_noisy)

    # Find the AAA transition matrix
    _, M = OptVariance(eps, x_grid, x_q_noisy)
    filename = 'data/%s_%s_M_%.2f_%d.csv'%(est_type,dist_type,eps,d)
    pd.DataFrame(M).to_csv(filename)
    print("AAA Solution Found!\n")

x= [0.   0.05 0.1  0.15 0.2  0.25 0.3  0.35 0.4  0.45 0.5  0.55 0.6  0.65
 0.7  0.75 0.8  0.85 0.9  0.95]
q(x)= [0.147 0.126 0.109 0.093 0.08  0.069 0.06  0.051 0.044 0.038 0.033 0.028
 0.024 0.021 0.018 0.015 0.013 0.011 0.01  0.008]
100000 data samples generated with exp distribution.

eps=0.50
Found matrix M for sw estimator!
Random num pools generated.
Distributed Estimation with sw complete with 10 repeats
q_est(x)= [[0.142 0.117 0.123 0.094 0.088 0.066 0.052 0.038 0.043 0.045 0.039 0.033
  0.023 0.023 0.018 0.012 0.01  0.01  0.011 0.012]]
AAA Solution Found!

eps=1.00
Found matrix M for sw estimator!
Random num pools generated.
Distributed Estimation with sw complete with 10 repeats
q_est(x)= [[0.147 0.125 0.102 0.098 0.079 0.069 0.062 0.055 0.042 0.032 0.036 0.029
  0.026 0.017 0.017 0.013 0.016 0.013 0.01  0.012]]
AAA Solution Found!

eps=1.50
Found matrix M for sw estimator!
Random num pools generated.
Distributed Estimation with sw complete with 10 repeats
q_est(x)= [[0.145 0